In [57]:
def main(imp,imp2,num_PLAY,price,dec):#decは選択数依存ならS  UCB依存ならU
    import json
    f=open('A-1.json')
    data=json.load(f)
    send_acts=treesearch(data,imp,imp2,num_PLAY,price,dec,data["turn"])
    print(send_acts)
if __name__=="__main__":
    from time import time
    t1=time()
    main(22,0.3,1000,10,"S")
    t2=time()
    print(t2-t1)

1
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 2, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
True
True
False
True
True
False
True
True
False
False
False
False
False
False
False
False
False
False
8
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 2, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
True
True
False
True
True
False
True
True
False
False
False
False
False
False
False
False
False
False
5
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 

IndexError: list index out of range

In [39]:
def get_max_sel(nodes):
    for j in range(9):
        if nodes[j]["isTrue"]==False:
            continue
        else:
            m=j
            break
    for s in range(j+1,9):
        if nodes[s]["isTrue"]==False:
            continue
        if nodes[m]["selected"]<nodes[s]["selected"]:
            m=s
    return m

In [56]:
def treesearch(data,imp,imp2,num_PLAY,price,dec,turn):
    import math
    num_agent=len(data["teams"][0]["agents"])#エージェントの数
    exe_turn=math.floor(24/num_agent)
    nodes=[]
    acts=[]
    send_acts=[]
    imp2=imp2/(turn+1-data["turn"])
    for i in range(num_agent):
        searcher(imp,imp2,num_PLAY,data,price,i,num_agent,nodes,exe_turn)
        if dec == "S":
            n=get_max_sel(nodes)
            s=get_max_sel(nodes[n]["child"])
            acts.append(n)
        elif dec == "U":
            n=get_max_ucb(nodes,num_PLAY,imp,imp2)
            acts.append(n)
            s=get_max_ucb(nodes[n]["child"],num_PLAY,imp,imp2)
        print(nodes[n]["teams"])
        nodes=nodes[n]["child"][s]["child"]
        print(n)
    for i in range(num_agent):
        act=get_act()
        act["agentID"]=data["teams"][0]["agents"][i]["agentID"]#場合によっては0を1に変更 
        if acts[i]<9:
            act["type"]="move"
            m_y=math.floor(acts[i]/3)-1
        elif acts[i]<18:
            act["type"]="remove"
            m_y=math.floor((acts[i]-9)/3)-1
        act["dx"]=(acts[i]%3)-1
        act["dy"]=m_y
        send_acts.append(act)
    return send_acts

In [18]:
def cir2(tiled,teamID):#囲み判定
    c=[]
    height=len(tiled)
    width=len(tiled[0])
    for i in range(height):
        o=[]
        for n in range(width):
            o.append(0)
        c.append(o)
    for i in range(height):
        for n in range(width):
            if tiled[i][n]==teamID:
                c[i][n]=9
                for k in range(width):
                    if k+1>n:
                        break
                    elif tiled[i][n-k-1]==teamID:
                        break
                    c[i][n-k-1]+=1
                for k in range(width):
                    if n+k+1>=width:
                        break
                    elif tiled[i][n+k+1]==teamID:
                        break
                    c[i][n+k+1]+=1
                for k in range(height):
                    if k+1>i:
                        break
                    elif tiled[i-k-1][n]==teamID:
                        break
                    c[i-k-1][n]+=1
                for k in range(height):
                    if i+k+1>=height:
                        break
                    elif tiled[i+k+1][n]==teamID:
                        break
                    c[i+k+1][n]+=1
    if not c[height-1][width-1]==9:
        c[height-1][width-1]=0
    if not c[height-1][0]==9:
        c[height-1][0]=0
    if not c[0][width-1]==9:
        c[0][width-1]=0
    if not c[0][0]==9:
        c[0][0]=0
    for s in range(width*height):
        count_exe=0
        for n in range(height):
            for i in range(width):
                if c[n][i]<4:
                    if n-1>=0:
                        if 0<c[n-1][i]<9:
                            c[n-1][i]=0
                            count_exe+=1
                    if i-1>=0:
                        if 0<c[n][i-1]<9:
                            c[n][i-1]=0
                            count_exe+=1
                    if n+1<len(c):
                        if 0<c[n+1][i]<9:
                            c[n+1][i]=0
                            count_exe+=1
                    if i+1<len(c[n]):
                        if 0<c[n][i+1]<9:
                            c[n][i+1]=0
                            count_exe+=1
        if count_exe==0:#1回も実行されなかったら終わる
            break
    return c

In [17]:
def reverse(tiled,teamID1,teamID2):
    for i in range(len(tiled)):
        for j in range(len(tiled[0])):
            if tiled[i][j]==teamID1:
                tiled[i][j]=teamID2
            elif tiled[i][j]==teamID2:
                tiled[i][j]=teamID1

In [15]:
def get_max_ucb(nodes,num_sel,imp,imp2):#最もUCBが大きいものの位置を返す
    from numpy import argmax 
    pos_ucb=0
    ucbs=[]
    for l in range(9):
        if nodes[l]["isTrue"]==False:
            ucbs.append(-1000)
            continue
        ucbs.append(get_UCB(imp,imp2,num_sel,nodes[l]["selected"],nodes[l]["per_win"],nodes[l]["score_d"]-0.5))
    pos_ucb = argmax(ucbs)
    return pos_ucb

In [14]:
def get_UCB(imp,imp2,num_sel,selected,Q,score_d):#score1　獲得スコア　UCBの計算 Q　勝率   imp2 スコアの差の重要性  score_d  スコアの差
    from math import sqrt
    from math import log
    if selected==0:
        return 10000
    return Q+imp*sqrt(2*log(num_sel)/selected)+imp2*score_d

In [ ]:
def move_pos_act(agent,act,tiled,agents,teamID_e,teamID_s):#agents　は味方だけ
    import math
    act=act-1
    m_x=(act%3)-1
    m_y=int(math.floor(act/3))-1
    if agent["x"]+m_x<=0 or agent["x"]+m_x>len(tiled[0]) or agent["y"]+m_y<=0 or agent["y"]+m_y>len(tiled):
        return False
    elif tiled[agent["y"]+m_y-1][agent["x"]+m_x-1]==teamID_e:
        return False
    elif tiled[agent["y"]+m_y-1][agent["x"]+m_x-1]==0:
        tiled[agent["y"]+m_y-1][agent["x"]+m_x-1]=teamID_s
    else:
        for i in agents:
            if agent["x"]+m_x==i["x"] and agent["y"]+m_y==i["y"]:
                return False
    agent["x"]=agent["x"]+m_x
    agent["y"]=agent["y"]+m_y
    return True

In [ ]:
def remove_act(agent,act,tiled,teams,teamID_e,teamID_s):
    import math
    act=act-10
    m_x=(act%3)-1
    m_y=int(math.floor(act/3))-1
    if m_x==0 and m_y == 0:
        return False
    if agent["x"]+m_x<=0 or agent["x"]+m_x>len(tiled[0]) or agent["y"]+m_y<=0 or agent["y"]+m_y>len(tiled):
        return False
    elif tiled[agent["y"]+m_y-1][agent["x"]+m_x-1]==0:
        return False
    elif  tiled[agent["y"]+m_y-1][agent["x"]+m_x-1]==teamID_e:
        for i in teams[1]["agents"]:
            if agent["x"]+m_x==i["x"] and agent["y"]+m_y==i["y"]:
                return False
        tiled[agent["y"]+m_y-1][agent["x"]+m_x-1]=0
    elif  tiled[agent["y"]+m_y-1][agent["x"]+m_x-1]==teamID_s:
        for i in teams[0]["agents"]:
            if agent["x"]+m_x==i["x"] and agent["y"]+m_y==i["y"]:
                return False
        tiled[agent["y"]+m_y-1][agent["x"]+m_x-1]=0
    return True

In [48]:
def judge_mT(agent,act,tiled,agents,teamID_e):
    import math
    act=act-1
    m_x=(act%3)-1
    m_y=int(math.floor(act/3))-1
    if m_x==0 and m_y==0:
        return True
    if (agent["x"]+m_x)<=0 or (agent["x"]+m_x)>len(tiled[0]) or (agent["y"]+m_y)<=0 or (agent["y"]+m_y)>len(tiled):
        return False
    elif tiled[agent["y"]+m_y-1][agent["x"]+m_x-1]==teamID_e:
        return False
    for i in agents:
        if agent["x"]+m_x==i["x"] and agent["y"]+m_y==i["y"]:
            return False
    return True

In [12]:
def judge_rT(agent,act,tiled,teams):
    import math
    act=act-10
    m_x=(act%3)-1
    m_y=int(math.floor(act/3))-1
    if m_x==0 and m_y == 0:
        return False
    if (agent["x"]+m_x)<=0 or (agent["x"]+m_x)>len(tiled[0]) or (agent["y"]+m_y)<=0 or (agent["y"]+m_y)>len(tiled):
        return False
    if tiled[agent["y"]+m_y-1][agent["x"]+m_x-1]==0:
        return False
    for j in range(2):
        for i in teams[j]["agents"]:
            if agent["x"]+m_x==i["x"] and agent["y"]+m_y==i["y"]:
                return False
    return True

In [29]:
def move(agent,act,tiled,teamID_s):
    import math
    act=act-1
    m_x=(act%3)-1
    m_y=int(math.floor(act/3))-1
    tiled[agent["y"]+m_y-1][agent["x"]+m_x-1]=teamID_s
    agent["x"]=agent["x"]+m_x
    agent["y"]=agent["y"]+m_y

In [30]:
def remove(agent,act,tiled):
    import math
    act=act-10
    m_x=(act%3)-1
    m_y=int(math.floor(act/3))-1
    tiled[agent["y"]+m_y-1][agent["x"]+m_x-1]=0

In [54]:
def searcher(imp,imp2,num_PLAY,data,price,move_agent_num,num_agent,nodes,exe_turn):#agent_numは味方想定 
    from copy import deepcopy
    num_sel=0
    for i in range(num_PLAY):
        if i == 0:
            if move_agent_num==0:
                for n in range(9):
                    if judge_mT(data["teams"][0]["agents"][0],n+1,data["tiled"],data["teams"][0]["agents"],data["teams"][1]["teamID"])==False:
                        nodes.append(get_False())
                        continue
                    else:
                        nodes.append(get_node(data["teams"][0]["tilePoint"]+data["teams"][0]["areaPoint"],n+1))
                        nodes[n]["tiled_acted"]=deepcopy(data["tiled"])
                        agent=deepcopy(data["teams"][0]["agents"][0])
                        culsco(data["points"],n+1,nodes[n],agent,data["teams"][0]["teamID"])
                        nodes[n]["score_d"]=nodes[n]["score_a"]-nodes[n]["score_e"]
                for n in range(9,18):
                    if judge_rT(data["teams"][0]["agents"][0],n+1,data["tiled"],data["teams"])==False:
                        nodes.append(get_False())
                        continue
                    else:
                        nodes.append(get_node2(data["teams"][1]["tilePoint"]+data["teams"][1]["areaPoint"],n+1))
                        nodes[n]["tiled_acted"]=deepcopy(data["tiled"])
                        agent=deepcopy(data["teams"][0]["agents"][0])
                        culsco2(data["points"],n+1,nodes[n],agent,data["teams"][0]["teamID"])
                        nodes[n]["score_d"]=nodes["score_a"]-["score_e"]
            else:
                for n in range(18):
                    if nodes[n]["isTrue"]==False:
                        continue
                    num_sel+=nodes[n]["selected"] 
        n=get_max_ucb(nodes,num_sel,imp,imp2)
        nodes[n]["selected"]+=1
        num_sel+=1
        if nodes[n]["selected"]>=price:
            if nodes[n]["selected"]==price:
                print(n)
                if n <9:
                    tiled=deepcopy(data["tiled"])
                    teams=deepcopy(data["teams"])
                    move(teams[0]["agents"][move_agent_num],n+1,tiled,teams[0]["teamID"])
                    for s in range(1,10):
                        if judge_mT(data["teams"][1]["agents"][move_agent_num],s,tiled,teams[1]["agents"],teams[0]["teamID"])==False:
                            nodes[n]["child"].append(get_False())
                            continue     
                        else:                        
                            nodes[n]["child"].append(get_node(nodes[n]["score_a"],s))
                            nodes[n]["child"][s-1]["tiled_acted"] = deepcopy(tiled)
                            nodes[n]["child"][s-1]["teams"] = deepcopy(teams)
                    for s in range(10,19):
                        if judge_rT(data["teams"][1]["agents"][move_agent_num],s,tiled,teams)==False:
                            nodes[n]["child"].append(get_False())
                            continue
                        else:
                            nodes[n]["child"].append(get_node2(nodes[n]["score_e"],s))
                            nodes[n]["child"][s-1]["tiled_acted"] = deepcopy(tiled)
                            nodes[n]["child"][s-1]["teams"] = deepcopy(teams)
                elif n <18:
                    tiled=deepcopy(data["tiled"])
                    teams=deepcopy(data["teams"])
                    remove(teams[0]["agents"][move_agent_num],n+1,tiled,teams)
                    for s in range(1,10):
                        if judge_mT(data["teams"][1]["agents"][move_agent_num],s,tiled,teams[1]["agents"],teams[0]["teamID"])==False:
                            nodes[n]["child"].append(get_False())
                            continue
                        else:
                            nodes[n]["child"].append(get_node(nodes[n]["score_a"],s))
                            nodes[n]["child"][s-1]["tiled_acted"] = deepcopy(tiled)
                            nodes[n]["child"][s-1]["teams"] = deepcopy(teams)
                    for s in range(10,19):
                        if judge_rT(data["teams"][1]["agents"][move_agent_num],s,tiled,teams)==False:
                            nodes[n]["child"].append(get_False())
                            continue
                        else:
                            nodes[n]["child"].append(get_node2(nodes[n]["score_e"],s))
                            nodes[n]["child"][s-1]["tiled_acted"] = deepcopy(tiled)
                            nodes[n]["child"][s-1]["teams"] = deepcopy(teams)
                print(nodes[n]["tiled_acted"])
                for i in range(18):
                    print(nodes[n]["child"][i]["isTrue"])
            else:
                winner=exe_enemy_node(data["points"],imp,imp2,price,nodes[n]["child"],nodes[n]["selected"]-price,num_agent,move_agent_num,0,exe_turn)
                nodes[n]["time_win"]+=winner
                nodes[n]["per_win"]=nodes[n]["time_win"]/nodes[n]["selected"]
                continue
        nodes[n]["time_win"]+=get_winner(nodes[n])
        nodes[n]["per_win"]=nodes[n]["time_win"]/nodes[n]["selected"]

In [33]:
def exe_enemy_node(points,imp,imp2,price,nodes,num_sel,num_agent,move_agent_num,turn_exe,exe_turn):
    from copy import deepcopy
    if num_sel==1:
        for n in range(9):
            if nodes[n]["isTrue"]==False:
                continue
            culsco(points,n+1,nodes[n],nodes[n]["teams"][1]["agents"][move_agent_num],nodes[n]["teams"][1]["teamID"])
            nodes[n]["score_d"]=nodes[n]["score_a"]-nodes[n]["score_e"]
        for n in range(9,18):
            if nodes[n]["isTrue"]==False:
                continue
            culsco2(points,n+1,nodes[n],nodes[n]["teams"][1]["agents"][move_agent_num],nodes[n]["teams"][1]["teamID"])
            nodes[n]["score_d"]=nodes[n]["score_a"]-nodes[n]["score_e"]
    n=get_max_ucb(nodes,num_sel,imp,imp2)
    nodes[n]["selected"]+=1
    if move_agent_num==num_agent:
        turn_exe+=1
        if turn_exe == exe_turn:
            winner=get_winner(nodes[n])
            nodes[n]["time_win"]+=winner
            nodes[n]["per_win"]=nodes[n]["time_win"]/nodes[n]["selected"]
            return 1.0-winner
            #勝率計算
    if nodes[n]["selected"]>=price:
        move_agent_num_n=(move_agent_num+1)%num_agent
        if nodes[n]["selected"]==price:
            winner=get_winner(nodes[n])
            if n<9:
                tiled=deepcopy(nodes[n]["tiled_acted"])
                teams=deepcopy(nodes[n]["teams"])
                move(teams[1]["agents"][move_agent_num],n+1,tiled,teams[1]["teamID"])
                for s in range(1,10):
                    if judge_mT(nodes[n]["teams"][0]["agents"][move_agent_num_n],s,tiled,teams[0]["agents"],teams[1]["teamID"])==False:
                            nodes[n]["child"].append(get_False())
                            continue     
                    else:                        
                        nodes[n]["child"].append(get_node(nodes[n]["score_a"],s))
                        nodes[n]["child"][s-1]["tiled_acted"] = deepcopy(tiled)
                        nodes[n]["child"][s-1]["teams"] = deepcopy(teams)
                        culsco(points,n+1,nodes[n]["child"][s-1],nodes[n]["child"][s-1]["teams"][0]["agents"][move_agent_num],nodes[n]["child"][s-1]["teams"][0]["teamID"])
                        nodes[n]["child"][s-1]["score_d"]=nodes[n]["child"][s-1]["score_a"]-nodes[n]["child"][s-1]["score_e"]
                for s in range(10,19):
                    if judge_rT(nodes[n]["teams"][0]["agents"][move_agent_num_n],s,tiled,teams)==False:
                        nodes[n]["child"].append(get_False())
                        continue
                    else:
                        nodes[n]["child"].append(get_node2(nodes[n]["score_e"],s))
                        nodes[n]["child"][s-1]["tiled_acted"] = deepcopy(tiled)
                        nodes[n]["child"][s-1]["teams"] = deepcopy(teams)
                        culsco2(points,n+1,nodes[n]["child"][s-1],nodes[n]["child"][s-1]["teams"][0]["agents"][move_agent_num],nodes[n]["child"][s-1]["teams"][0]["teamID"])
                        nodes[n]["child"][s-1]["score_d"]=nodes[n]["child"][s-1]["score_a"]-nodes[n]["child"][s-1]["score_e"]
            elif n<18:
                tiled=deepcopy(nodes[n]["tiled_acted"])
                teams=deepcopy(nodes[n]["teams"])
                remove(teams[1]["agents"][move_agent_num],n+1,tiled,teams)
                for s in range(1,10):
                    if judge_mT(nodes[n]["teams"][0]["agents"][move_agent_num_n],s,tiled,teams[0]["agents"],teams[1]["teamID"])==False:
                        nodes[n]["child"].append(get_False())
                        continue
                    else:
                        nodes[n]["child"].append(get_node(nodes[n]["score_a"],s))
                        nodes[n]["child"][s-1]["tiled_acted"] = deepcopy(tiled)
                        nodes[n]["child"][s-1]["teams"] = deepcopy(teams)
                        culsco(points,n+1,nodes[n]["child"][s-1],nodes[n][child][s-1]["teams"][0]["agents"][move_agent_num],nodes[n]["child"][s-1]["teams"][0]["teamID"])
                        nodes[n]["child"][s-1]["score_d"]=nodes[n]["child"][s-1]["score_a"]-nodes[n]["child"][s-1]["score_e"]
                for s in range(10,19):
                    if judge_rT(nodes[n]["teams"][0]["agents"][move_agent_num_n],s,tiled,teams)==False:
                        nodes[n]["child"].append(get_False())
                        continue
                    else:
                        nodes[n]["child"].append(get_node2(nodes[n]["score_e"],s))
                        nodes[n]["child"][s-1]["tiled_acted"] = deepcopy(tiled)
                        nodes[n]["child"][s-1]["teams"] = deepcopy(teams)
                        culsco2(points,n+1,nodes[n]["child"][s-1],nodes[n]["child"][s-1]["teams"][0]["agents"][move_agent_num],nodes[n]["child"][s-1]["teams"][0]["teamID"])
                        nodes[n]["child"][s-1]["score_d"]=nodes[n]["child"][s-1]["score_a"]-nodes[n]["child"][s-1]["score_e"]
        else:
            winner=exe_side_node(points,imp,imp2,price,nodes[n]["child"],nodes[n]["selected"]-price,num_agent,move_agent_num_n,turn_exe,exe_turn)
    else:             
        winner=get_winner(nodes[n])
    nodes[n]["time_win"]+=winner
    nodes[n]["per_win"]=nodes[n]["time_win"]/nodes[n]["selected"]
    return 1.0-winner

In [7]:
def exe_side_node(points,imp,imp2,price,nodes,num_sel,num_agent,move_agent_num,turn_exe,exe_turn): 
    from copy import deepcopy
    n=get_max_ucb(nodes,num_sel,imp,imp2)
    nodes[n]["selected"]+=1
    if nodes[n]["selected"]>=price:
        if nodes[n]["selected"]==price:
            winner=get_winner(nodes[n])
            if n<9:
                tiled=deepcopy(nodes[n]["tiled_acted"])
                teams=deepcopy(nodes[n]["teams"])
                move(teams[0]["agents"][move_agent_num],n+1,tiled,teams[0]["teamID"])
                for s in range(1,10):
                    if judge_mT(nodes[n]["teams"][1]["agents"][move_agent_num],s,tiled,teams[1]["agents"],teams[0]["teamID"])==False:
                        nodes[n]["child"].append(get_False())
                        continue
                    else:
                        nodes[n]["child"].append(get_node(nodes[n]["score_a"],s))
                        nodes[n]["child"][s-1]["tiled_acted"] = deepcopy(tiled)
                        nodes[n]["child"][s-1]["teams"] = deepcopy(teams)
                for s in range(10,19):
                    if judge_rT(nodes[n]["teams"][1]["agents"][move_agent_num],s,tiled,teams)==False:
                        nodes[n]["child"].append(get_False())
                        continue
                    else:
                        nodes[n]["child"].append(get_node2(nodes[n]["score_e"],s))
                        nodes[n]["child"][s-1]["tiled_acted"] = deepcopy(tiled)
                        nodes[n]["child"][s-1]["teams"] = deepcopy(teams)
            elif n<18:
                tiled=deepcopy(nodes[n]["tiled_acted"])
                teams=deepcopy(nodes[n]["teams"])
                remove(teams[0]["agents"][move_agent_num],n+1,tiled,teams)
                for s in range(1,10):
                    if judge_mT(nodes[n]["teams"][1]["agents"][move_agent_num],s,tiled,teams[1]["agents"],teams[0]["teamID"])==False:
                        nodes[n]["child"].append(get_False())
                        continue
                    else:
                        nodes[n]["child"].append(get_node(nodes[n]["score_a"],s))
                        nodes[n]["child"][s-1]["tiled_acted"] = deepcopy(tiled)
                        nodes[n]["child"][s-1]["teams"] = deepcopy(teams)
                for s in range(10,19):
                    if judge_rT(nodes[n]["teams"][1]["agents"][move_agent_num_n],s,tiled,teams)==False:
                        nodes[n]["child"].append(get_False())
                        continue
                    else:
                        nodes[n]["child"].append(get_node2(nodes[n]["score_e"],s))
                        nodes[n]["child"][s-1]["tiled_acted"] = deepcopy(tiled)
                        nodes[n]["child"][s-1]["teams"] = deepcopy(teams)
        else:
            winner=exe_enemy_node(points,imp,imp2,price,nodes[n]["child"],nodes[n]["selected"]-price,num_agent,move_agent_num,turn_exe,exe_turn)
    else: 
        winner=get_winner(nodes[n])
    nodes[n]["time_win"]+=winner
    nodes[n]["per_win"]=nodes[n]["time_win"]/nodes[n]["selected"]
    return 1.0-winner 

In [6]:
def get_False():
    return {
        "isTrue":False
    }

In [5]:
def get_node(score,act):
    node={
        "isTrue":True,
        "action": act,
        "time_win":0,
        "score_e":score,#敵のスコア
        "score_a":0,#行動後の自分のスコア
        "score_d":0,
        "per_win":0,
        "selected":0,
        "tiled_acted":None,#前のタイルの行動後の状態(子にとってみると現在の状態)
        "teams":None,#敵の行動後の状態（現在状態）
        "child":[]
    }
    return node

In [4]:
def get_node2(score,act):
    #get_node2(nodes[n]["score_e"],s)
    node={
        "isTrue":True,
        "action": act,
        "time_win":0,
        "score_e":0,#敵のスコア
        "score_a":score,#行動後の自分のスコア
        "score_d":0,
        "per_win":0,
        "selected":0,
        "tiled_acted":None,#前のタイルの行動後の状態(子にとってみると現在の状態)
        "teams":None,#敵の行動後の状態（現在状態）
        "child":[]
    }
    return node

In [3]:
def culsco(points,act,node,agent,teamID):
    from copy import deepcopy
    tile=deepcopy(node["tiled_acted"])
    agent_t=deepcopy(agent)
    move(agent,act,tile,teamID)
    tl=cir2(tile,teamID)
    score=0
    for i in range(len(tile)):
        for j in range(len(tile[0])):
            if tl[i][j]==4:
                score+=abs(points[i][j])
            if tl[i][j]==9:
                score+=points[i][j]
    node["score_a"]=score

In [2]:
def culsco2(points,act,node,agent,teamID):
    from copy import deepcopy
    tile=deepcopy(node["tiled_acted"])
    agent_t=deepcopy(agent)
    remove(agent_t,act,tile)
    tl=cir2(tile,teamID)
    score=0
    for i in range(len(tile)):
        for j in range(len(tile[0])):
            if tl[i][j]==4:
                score+=abs(points[i][j])
            if tl[i][j]==9:
                score+=points[i][j]
    node["score_e"]=score

In [1]:
def get_winner(node):
    if node["score_e"]>node["score_a"]:
        return 0
    elif node["score_e"]<node["score_a"]:
        return 1
    else:
        return 0.5